In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


Mounted at /gdrive
/gdrive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 64.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9353587cfee78774e978b83d85d0f4bca0949ade392c349f86ef1446a2aca4f5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *


In [ ]:
spark = SparkSession.Builder().getOrCreate()


In [ ]:
data = spark.read.csv('/gdrive/MyDrive/cc_data.csv',header = True,inferSchema=True)


In [ ]:
data.show()


+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|CUST_ID|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
| C10001|  40.900749|         0.818182|     95.4|             0.0|                  95.4|  

In [ ]:
data.limit(5).show()


+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
|CUST_ID|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|
+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+
| C10001|  40.900749|         0.818182|     95.4|             0.0|                  95.4|  

In [ ]:
data.limit(5).toPandas()


,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [ ]:
data = data.na.drop(how='any')
# create a new column where purchase frequency >=0.5 - > 1 and rest all termed as 0 
data = data.withColumn("Label",data.PURCHASES_FREQUENCY>=0.5)
# casting of the label into string
data = data.withColumn("Label",data["Label"].cast("String"))
print(data.show())

# string indexer 
from pyspark.ml.feature import StringIndexer
index = StringIndexer(inputCol = "Label",outputCol = "Target")
train = index.fit(data).transform(data)

train.show()


+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+-----+
|CUST_ID|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|Label|
+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+-----+
| C10001|  40.900749|         0.818182|     95.4|             0.0|       

In [ ]:
from pyspark.ml.feature import VectorAssembler
VecAssemble = VectorAssembler(inputCols=['BALANCE','BALANCE_FREQUENCY','PURCHASES','ONEOFF_PURCHASES','INSTALLMENTS_PURCHASES',
                                         'CASH_ADVANCE','ONEOFF_PURCHASES_FREQUENCY','PURCHASES_INSTALLMENTS_FREQUENCY','CASH_ADVANCE_FREQUENCY','CASH_ADVANCE_TRX','PURCHASES_TRX','CREDIT_LIMIT','PAYMENTS','MINIMUM_PAYMENTS','PRC_FULL_PAYMENT','TENURE'],outputCol='Features')
feat_data = VecAssemble.transform(train)

feat_data = feat_data.select('Features','Target')
print(feat_data.show(10))



+--------------------+------+
|            Features|Target|
+--------------------+------+
|[40.900749,0.8181...|   1.0|
|[3202.467416,0.90...|   1.0|
|[2495.148862,1.0,...|   0.0|
|[817.714335,1.0,1...|   1.0|
|[1809.828751,1.0,...|   0.0|
|[627.260806,1.0,7...|   0.0|
|[1823.652743,1.0,...|   0.0|
|[1014.926473,1.0,...|   1.0|
|[152.225975,0.545...|   1.0|
|[1293.124939,1.0,...|   0.0|
+--------------------+------+
only showing top 10 rows

None


In [ ]:
#split the data 
ratio = [0.8,0.2]
TrainData,TestData = feat_data.randomSplit(ratio,2)
TrainData.show(10)

+--------------------+------+
|            Features|Target|
+--------------------+------+
|(16,[0,1,2,3,6,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
|(16,[0,1,5,8,9,11...|   1.0|
+--------------------+------+
only showing top 10 rows



In [ ]:
from pyspark.ml.classification import LogisticRegression
lrModel = LogisticRegression(featuresCol='Features',labelCol='Target',maxIter=20)
lrModel = lrModel.fit(TrainData)
history = lrModel.summary
print(history.accuracy)

0.9794138670478914


In [ ]:
op = lrModel.transform(TestData)
op.show(15)

+--------------------+------+--------------------+--------------------+----------+
|            Features|Target|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(16,[0,1,5,8,9,11...|   1.0|[-14.507644462953...|[5.00506644047141...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-15.139086076177...|[2.66181789515123...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-17.714816373892...|[2.02559675263364...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-13.850293960011...|[9.65813654269541...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-15.763546260195...|[1.42553794565962...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-14.408897792393...|[5.52452519476443...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-16.081163925531...|[1.03762206754217...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-13.704427777154...|[1.11748616076214...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[-15.708001071791...|[1.50696007833078...|       1.0|
|(16

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [ ]:
eval = BinaryClassificationEvaluator(labelCol = 'Target',rawPredictionCol='prediction')
print("Testing Accuracy: ", eval.evaluate(op))


Testing Accuracy:  0.9770835502536107


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:

rfmodel = RandomForestClassifier(featuresCol='Features',labelCol = 'Target')

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder().addGrid(rfmodel.numTrees,[100,200,300])\
            .addGrid(rfmodel.maxDepth,[2,3,4,7])\
            .addGrid(rfmodel.impurity,["entropy","gini"]).build()

    

In [63]:
eva11 = BinaryClassificationEvaluator(labelCol="Target", rawPredictionCol="prediction")

crossval = CrossValidator(estimator = rfmodel,
                          estimatorParamMaps=paramGrid,
                          evaluator=eva11,
                          numFolds=5)


In [64]:
finalModel = crossval.fit(TrainData)
print(finalModel)

CrossValidatorModel_745bfd6dd464


In [65]:
predictions = finalModel.transform(TestData)

In [66]:
print(predictions.show())

+--------------------+------+--------------------+--------------------+----------+
|            Features|Target|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(16,[0,1,5,8,9,11...|   1.0|[0.26511302867479...|[0.00132556514337...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.21768482510628...|[0.00108842412553...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.15690445859868...|[7.84522292993443...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.17753795183741...|[8.87689759187063...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.20638157509873...|[0.00103190787549...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.18205876639477...|[9.10293831973856...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.24089625470314...|[0.00120448127351...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.15628618668375...|[7.81430933418766...|       1.0|
|(16,[0,1,5,8,9,11...|   1.0|[0.21031471177523...|[0.00105157355887...|       1.0|
|(16

In [67]:
eval2 = BinaryClassificationEvaluator(labelCol='Target',rawPredictionCol='prediction')
eval2.evaluate(predictions)

0.9824621814990554